In [1]:
from fairseq.models.roberta import CamembertModel
camembert = CamembertModel.from_pretrained('./Camembert-base/')
camembert.eval()
masked_line = 'Le camembert est <mask> :)'
camembert.fill_mask(masked_line, topk=3)

# The underlying model is available under the *models* attribute


/Users/Jorg/miniconda3/envs/CamemBERT/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-09-08 15:05:37 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


ImportError: dlopen(/Users/Jorg/miniconda3/envs/CamemBERT/lib/python3.9/site-packages/lxml/etree.cpython-39-darwin.so, 0x0002): tried: '/Users/Jorg/miniconda3/envs/CamemBERT/lib/python3.9/site-packages/lxml/etree.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have (x86_64), need (arm64e)))

In [2]:
import torch

from transformers.modeling_camembert import CamembertForMaskedLM
from transformers.tokenization_camembert import CamembertTokenizer


def fill_mask(masked_input, model, tokenizer, topk=5):
    # Adapted from https://github.com/pytorch/fairseq/blob/master/fairseq/models/roberta/hub_interface.py
    assert masked_input.count("<mask>") == 1
    input_ids = torch.tensor(tokenizer.encode(masked_input, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    logits = model(input_ids)[0]  # The last hidden-state is the first element of the output tuple
    masked_index = (input_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()
    logits = logits[0, masked_index, :]
    prob = logits.softmax(dim=0)
    values, indices = prob.topk(k=topk, dim=0)
    topk_predicted_token_bpe = " ".join(
        [tokenizer.convert_ids_to_tokens(indices[i].item()) for i in range(len(indices))]
    )
    masked_token = tokenizer.mask_token
    topk_filled_outputs = []
    for index, predicted_token_bpe in enumerate(topk_predicted_token_bpe.split(" ")):
        predicted_token = predicted_token_bpe.replace("\u2581", " ")
        if " {0}".format(masked_token) in masked_input:
            topk_filled_outputs.append(
                (
                    masked_input.replace(" {0}".format(masked_token), predicted_token),
                    values[index].item(),
                    predicted_token,
                )
            )
        else:
            topk_filled_outputs.append(
                (masked_input.replace(masked_token, predicted_token), values[index].item(), predicted_token,)
            )
    return topk_filled_outputs


tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")
model.eval()

masked_input = "Le camembert est <mask> :)"
print(fill_mask(masked_input, model, tokenizer, topk=3))

ModuleNotFoundError: No module named 'transformers.modeling_camembert'

In [2]:
from transformers import RobertaConfig, RobertaModel
import pandas as pd

csv = pd.read_csv('./Post_Data/Training_data.csv')

# Initializing a RoBERTa configuration
configuration = RobertaConfig()

# Initializing a model from the configuration
model = RobertaModel(configuration)

# Accessing the model configuration
configuration = model.config

from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
print(tokenizer("Hello world")['input_ids'])
print(tokenizer(" Hello world")['input_ids'])
print(csv)

for i in csv['text']:
    print(tokenizer(i)['input_ids'])
    

[0, 31414, 232, 2]
[0, 20920, 232, 2]
                                                  text  label culture  \
0         eclairage de la gaine  à compléter rappel  2   SH16   fr-fr   
1    le boitier métallique sur le toit de cabine es...  CAR23   fr-fr   
2    plaque d  identification n  est pas complète o...  CAR26   fr-fr   
3    le bon fonctionnement du système appel de seco...  CAR01   fr-fr   
4    les données du propriétaire sont à placer en c...  CAR06   fr-fr   
..                                                 ...    ...     ...   
237  dossier de sécurité  le rapport d  analyse de ...  DOC02   fr-fr   
238            la vitre de la boîte à clé est absente   CAR19   fr-fr   
239                 nettoyer la cuvette 1 e répétition   SH01   fr-fr   
240  veuillez fixer les 3 poids dont la taille ne p...   CW01   fr-fr   
241  dossier de sécurité  les documents concernant ...    MOD   fr-fr   

                  document_id             annotation_id    source  
0    62baee150c69

In [10]:
from transformers import CamembertTokenizer, CamembertForTokenClassification
import torch

#tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertForTokenClassification.from_pretrained('camembert-base')



Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream tas

In [11]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")


##### Process text sample (from wikipedia)

from transformers import pipeline

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
nlp("Apple est créée le 1er avril 1976 dans le garage de la maison d'enfance de Steve Jobs à Los Altos en Californie par Steve Jobs, Steve Wozniak et Ronald Wayne14, puis constituée sous forme de société le 3 janvier 1977 à l'origine sous le nom d'Apple Computer, mais pour ses 30 ans et pour refléter la diversification de ses produits, le mot « computer » est retiré le 9 janvier 2015.")


ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.